<a href="https://colab.research.google.com/github/bkpearse/Cooking-Project/blob/benjamain/Whoosh_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

In [ ]:
pip install whoosh

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from whoosh.fields import *
from whoosh.index import create_in, open_dir
from whoosh.qparser import QueryParser
import whoosh.index as index
from whoosh.fields import Schema, TEXT
from whoosh.qparser import MultifieldParser
from whoosh.qparser import OrGroup
from whoosh.fields import Schema, KEYWORD
from whoosh.query import Phrase
from whoosh import qparser
from whoosh.query import Or
from whoosh.query import Term, And, BooleanQuery
from whoosh.index import open_dir
from whoosh import scoring
from whoosh import index

In [5]:
df = pd.read_csv('Data/RecipeNLG_dataset.csv')

In [6]:
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [90]:
# Define the schema of the index
my_schema = Schema(title=TEXT(stored=True), 
                ingredients=KEYWORD(stored=True, commas=True),
                directions=TEXT(stored=True),
                link=ID(stored=True),
                source=TEXT(stored=True),
                NER=TEXT(stored=True))

In [91]:
# Create the index or open it if it already exists
if not os.path.exists("new_index"):
    os.mkdir("new_index")
    ix = index.create_in("new_index", my_schema)
else:
    ix = index.open_dir("new_index")

In [92]:

# Set the chunk size
chunk_size = 10000

In [93]:
# Index the dataset in chunks
writer = ix.writer()
with open('Data/RecipeNLG_dataset.csv') as f:
    next(f)  # Skip the header row
    lines = []
    for line in f:
        line = line.strip().split(',')
        if len(line) == 7:
            lines.append(line)
        if len(lines) == chunk_size:
            for l in lines:
                writer.add_document(title=l[1], 
                                    ingredients=l[2], 
                                    directions=l[3], 
                                    link=l[4], 
                                    source=l[5], 
                                    NER=l[6])
            lines = []
            writer.commit()
            writer = ix.writer()
    # Add any remaining lines
    for l in lines:
        writer.add_document(title=l[1], 
                            ingredients=l[2], 
                            directions=l[3], 
                            link=l[4], 
                            source=l[5], 
                            NER=l[6])
    writer.commit()
    
print("Indexing complete!")

Indexing complete!


Defining the search term and running the search

In [103]:
# Define the search term
search_term = "Fresh chili peppers"

In [104]:
# Create a QueryParser for the "NER" field
qp = QueryParser("NER", schema=ix.schema)

In [105]:
# Parse the search term
q = qp.parse(search_term)

In [107]:
# Search the index and get the results
with ix.searcher() as searcher:
    results = searcher.search(q)
    
    # Print the results
    for result in results:
        print(result)

<Hit {'NER': '"[""Fresh chili peppers""]"', 'directions': '"[""Preheat oven to 500 degrees.""]"', 'ingredients': '"[""Fresh chili peppers""]"', 'link': 'www.cookbooks.com/Recipe-Details.aspx?id=117013', 'source': 'Gathered', 'title': 'Roasted Chili Peppers(To Roast And Peel Green Chilies)  '}>


Viewing our data in the index

In [94]:
with ix.searcher() as searcher:
    # Get the first document in the index
    docnum = 9
    doc = searcher.stored_fields(docnum)

    # Print the stored fields for the document
    print(doc)

{'NER': '"[""Fresh chili peppers""]"', 'directions': '"[""Preheat oven to 500 degrees.""]"', 'ingredients': '"[""Fresh chili peppers""]"', 'link': 'www.cookbooks.com/Recipe-Details.aspx?id=117013', 'source': 'Gathered', 'title': 'Roasted Chili Peppers(To Roast And Peel Green Chilies)  '}


In [95]:
with ix.searcher() as searcher:
  docnum = 9
  doc = searcher.stored_fields(docnum)

  ner = doc['NER']
  print(ner)

"[""Fresh chili peppers""]"
